In [2]:

# ============================================================

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---------------------------
# 0) Repro + folders
# ---------------------------
SEED = 42
rng = np.random.default_rng(SEED)

ROOT = os.getcwd()  # Colab => /content
DATA_DIR = os.path.join(ROOT, "data")
OUT_DIR = os.path.join(ROOT, "outputs")
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(OUT_DIR, exist_ok=True)

print("ROOT:", ROOT)
print("DATA_DIR:", DATA_DIR)
print("OUT_DIR:", OUT_DIR)

# ---------------------------
# 1) Synthetic Data Generator
# ---------------------------
def make_merchants(n_merchants=250):
    merchant_id = [f"M{str(i).zfill(4)}" for i in range(1, n_merchants + 1)]

    tenure_months = rng.integers(1, 60, size=n_merchants)  # 1 to 59
    risk_tier = rng.choice(["Low", "Medium", "High"], size=n_merchants, p=[0.55, 0.30, 0.15])

    # baseline monthly volume correlated with risk tier slightly
    base_volume = rng.lognormal(mean=10.2, sigma=0.55, size=n_merchants)  # ~$27k median-ish
    tier_mult = np.where(risk_tier == "High", 1.15, np.where(risk_tier == "Medium", 1.00, 0.90))
    monthly_volume_usd = np.round(base_volume * tier_mult, 2)

    # refund rate: higher in some risk tiers and lower tenure (slightly)
    refund_base = rng.beta(a=2.2, b=18.0, size=n_merchants)  # ~10% typical
    refund_rate = np.clip(refund_base + (risk_tier == "High") * 0.03 - (tenure_months / 120) * 0.01, 0.0, 0.35)

    merchant_df = pd.DataFrame({
        "merchant_id": merchant_id,
        "risk_tier": risk_tier,
        "tenure_months": tenure_months,
        "monthly_volume_usd": monthly_volume_usd,
        "refund_rate": np.round(refund_rate, 4)
    })
    return merchant_df


def make_transactions(merchant_df, n_txn=120_000, start="2024-01-01", end="2025-12-31"):
    # date range
    start_dt = pd.Timestamp(start)
    end_dt = pd.Timestamp(end)
    days = (end_dt - start_dt).days + 1
    txn_dates = start_dt + pd.to_timedelta(rng.integers(0, days, size=n_txn), unit="D")

    merchants = merchant_df["merchant_id"].to_numpy()
    # Weighted by monthly volume (bigger merchants = more transactions)
    w = merchant_df["monthly_volume_usd"].to_numpy()
    w = w / w.sum()
    txn_merchant = rng.choice(merchants, size=n_txn, p=w)

    channel = rng.choice(["CNP", "CP"], size=n_txn, p=[0.78, 0.22])
    country = rng.choice(["US", "CA", "UK", "IN", "AU", "BR", "MX", "DE"], size=n_txn,
                         p=[0.55, 0.07, 0.07, 0.08, 0.05, 0.06, 0.06, 0.06])

    # Amounts: lognormal but capped
    amt = rng.lognormal(mean=3.6, sigma=0.75, size=n_txn)  # ~ $36 median-ish
    amt = np.clip(amt, 2, 800)
    amount = np.round(amt, 2)

    txn_df = pd.DataFrame({
        "txn_id": [f"T{str(i).zfill(7)}" for i in range(1, n_txn + 1)],
        "txn_date": pd.to_datetime(txn_dates),
        "merchant_id": txn_merchant,
        "channel": channel,
        "country": country,
        "amount_usd": amount
    })

    return txn_df


def make_disputes(txn_df, merchant_df, dispute_rate_base=0.012):
    # Merge merchant features into txn-level for rate modeling
    m = merchant_df.set_index("merchant_id")
    df = txn_df.join(m, on="merchant_id")

    # Probability model (simple but realistic directional effects):
    # - higher for High risk tier
    # - higher for CNP
    # - higher for higher amounts (small effect)
    # - higher if refund_rate high (friendly fraud / operational patterns)
    tier_adj = np.where(df["risk_tier"].values == "High", 0.012,
                np.where(df["risk_tier"].values == "Medium", 0.004, -0.002))
    channel_adj = np.where(df["channel"].values == "CNP", 0.006, -0.003)
    amt_adj = np.clip((df["amount_usd"].values - 50) / 5000, -0.002, 0.004)
    refund_adj = np.clip((df["refund_rate"].values - 0.10) * 0.08, -0.004, 0.010)
    tenure_adj = np.clip((12 - df["tenure_months"].values) / 600, -0.004, 0.006)  # newer => slightly higher

    p = dispute_rate_base + tier_adj + channel_adj + amt_adj + refund_adj + tenure_adj
    p = np.clip(p, 0.001, 0.09)

    is_dispute = rng.random(len(df)) < p
    dispute_txns = df.loc[is_dispute].copy()

    # Reason codes distribution changes by channel/tier a bit
    reason_pool = ["FRAUD", "NO_AUTH", "NOT_RECEIVED", "NOT_AS_DESCRIBED", "DUPLICATE", "CANCELLED", "PROCESSING_ERROR"]
    # Base probabilities
    base_probs = np.array([0.23, 0.17, 0.18, 0.20, 0.07, 0.07, 0.08])

    # Adjust probabilities for CNP and High tier
    adj = []
    for _, r in dispute_txns.iterrows():
        probs = base_probs.copy()
        if r["channel"] == "CNP":
            probs[0] += 0.07  # FRAUD
            probs[2] += 0.03  # NOT_RECEIVED
            probs[5] -= 0.02  # CANCELLED
        if r["risk_tier"] == "High":
            probs[0] += 0.04
            probs[1] += 0.02
            probs[6] -= 0.02
        probs = np.clip(probs, 0.01, None)
        probs = probs / probs.sum()
        adj.append(probs)

    adj = np.vstack(adj)
    reason_idx = [rng.choice(len(reason_pool), p=adj[i]) for i in range(adj.shape[0])]
    dispute_txns["reason_code"] = [reason_pool[i] for i in reason_idx]

    # Evidence quality score (0-100): better for older merchants + lower risk
    eq = (
        55
        + (dispute_txns["tenure_months"].values / 60) * 25
        - (dispute_txns["risk_tier"].values == "High") * 10
        + rng.normal(0, 12, size=len(dispute_txns))
    )
    dispute_txns["evidence_quality"] = np.clip(eq, 0, 100).round(0).astype(int)

    # Response time days: faster for better ops; slower for high risk / new merchants
    rt = (
        7
        - (dispute_txns["tenure_months"].values / 60) * 2
        + (dispute_txns["risk_tier"].values == "High") * 2
        + rng.normal(0, 2.5, size=len(dispute_txns))
    )
    dispute_txns["response_time_days"] = np.clip(rt, 0, 30).round(0).astype(int)

    # Win probability model:
    # - Higher evidence quality
    # - Lower response time
    # - Certain reason codes easier/harder (e.g., processing errors easier to win)
    reason_win_adj = dispute_txns["reason_code"].map({
        "PROCESSING_ERROR": 0.18,
        "DUPLICATE": 0.12,
        "CANCELLED": 0.08,
        "NOT_AS_DESCRIBED": -0.05,
        "NOT_RECEIVED": -0.03,
        "NO_AUTH": -0.08,
        "FRAUD": -0.12
    }).values

    z = (
        -0.3
        + (dispute_txns["evidence_quality"].values - 60) / 25
        - (dispute_txns["response_time_days"].values) / 18
        + reason_win_adj
        - (dispute_txns["risk_tier"].values == "High") * 0.10
    )
    win_prob = 1 / (1 + np.exp(-z))
    win_prob = np.clip(win_prob, 0.02, 0.95)

    outcome = np.where(rng.random(len(dispute_txns)) < win_prob, "WIN", "LOSS")
    dispute_txns["outcome"] = outcome

    # Dispute amounts: usually full amount, sometimes partial
    partial = rng.random(len(dispute_txns)) < 0.12
    dispute_amt = np.where(partial, dispute_txns["amount_usd"].values * rng.uniform(0.3, 0.9, size=len(dispute_txns)),
                           dispute_txns["amount_usd"].values)
    dispute_txns["dispute_amount_usd"] = np.round(dispute_amt, 2)

    # Fees: simplified fixed + % (only on loss)
    fee = 15 + 0.01 * dispute_txns["dispute_amount_usd"].values
    dispute_txns["cb_fee_usd"] = np.round(fee, 2)

    # Final disputes table
    disputes = dispute_txns[[
        "txn_id", "txn_date", "merchant_id", "channel", "country",
        "amount_usd", "reason_code", "dispute_amount_usd",
        "evidence_quality", "response_time_days", "outcome", "cb_fee_usd",
        "risk_tier", "tenure_months", "monthly_volume_usd", "refund_rate"
    ]].copy()

    disputes.insert(0, "dispute_id", [f"D{str(i).zfill(7)}" for i in range(1, len(disputes) + 1)])
    disputes.rename(columns={"txn_date": "dispute_open_date"}, inplace=True)

    return disputes


# ---------------------------
# 2) Build data
# ---------------------------
merchants = make_merchants(n_merchants=250)
transactions = make_transactions(merchants, n_txn=120_000, start="2024-01-01", end="2025-12-31")
disputes = make_disputes(transactions, merchants, dispute_rate_base=0.012)

# Save raw data
merchants.to_csv(os.path.join(DATA_DIR, "merchants.csv"), index=False)
transactions.to_csv(os.path.join(DATA_DIR, "transactions.csv"), index=False)
disputes.to_csv(os.path.join(DATA_DIR, "disputes.csv"), index=False)

print("\nSaved raw data:")
print(" - data/merchants.csv")
print(" - data/transactions.csv")
print(" - data/disputes.csv")
print(f"\nCounts => merchants={len(merchants):,} txns={len(transactions):,} disputes={len(disputes):,}")

# ---------------------------
# 3) Analytics
# ---------------------------

# Helper: month bucket
transactions["month"] = transactions["txn_date"].dt.to_period("M").astype(str)
disputes["month"] = pd.to_datetime(disputes["dispute_open_date"]).dt.to_period("M").astype(str)

# 3.1 Chargeback rate & loss analysis (merchant-month)
txn_mm = transactions.groupby(["merchant_id", "month"], as_index=False).agg(
    txn_count=("txn_id", "count"),
    txn_amount=("amount_usd", "sum")
)

disp_mm = disputes.groupby(["merchant_id", "month"], as_index=False).agg(
    dispute_count=("dispute_id", "count"),
    dispute_amount=("dispute_amount_usd", "sum"),
    loss_amount=("dispute_amount_usd", lambda s: s.sum()),  # placeholder; will adjust below
    loss_fees=("cb_fee_usd", "sum")
)

# True losses only when outcome == LOSS
loss_mm = disputes[disputes["outcome"] == "LOSS"].groupby(["merchant_id", "month"], as_index=False).agg(
    loss_amount=("dispute_amount_usd", "sum"),
    loss_fees=("cb_fee_usd", "sum")
)

mm = txn_mm.merge(disp_mm[["merchant_id", "month", "dispute_count", "dispute_amount"]], on=["merchant_id", "month"], how="left")
mm = mm.merge(loss_mm, on=["merchant_id", "month"], how="left")
mm[["dispute_count", "dispute_amount", "loss_amount", "loss_fees"]] = mm[["dispute_count", "dispute_amount", "loss_amount", "loss_fees"]].fillna(0)

mm = mm.merge(merchants, on="merchant_id", how="left")
mm["chargeback_rate"] = mm["dispute_count"] / mm["txn_count"]
mm["net_loss_usd"] = mm["loss_amount"] + mm["loss_fees"]
mm["net_loss_rate"] = np.where(mm["txn_amount"] > 0, mm["net_loss_usd"] / mm["txn_amount"], 0)

mm.to_csv(os.path.join(OUT_DIR, "merchant_month_risk_metrics.csv"), index=False)

# 3.2 Dispute reason mapping (overall + by tier)
reason_overall = disputes.groupby("reason_code", as_index=False).agg(
    disputes=("dispute_id", "count"),
    dispute_amount=("dispute_amount_usd", "sum"),
    win_rate=("outcome", lambda x: (x == "WIN").mean())
).sort_values("disputes", ascending=False)

reason_tier = disputes.groupby(["risk_tier", "reason_code"], as_index=False).agg(
    disputes=("dispute_id", "count"),
    dispute_amount=("dispute_amount_usd", "sum"),
    win_rate=("outcome", lambda x: (x == "WIN").mean())
).sort_values(["risk_tier", "disputes"], ascending=[True, False])

reason_overall.to_csv(os.path.join(OUT_DIR, "dispute_reason_overall.csv"), index=False)
reason_tier.to_csv(os.path.join(OUT_DIR, "dispute_reason_by_tier.csv"), index=False)

# 3.3 Win/Loss drivers (simple explainable slices)
# Evidence quality buckets + response time buckets
disputes["evidence_bucket"] = pd.cut(disputes["evidence_quality"], bins=[-1, 40, 60, 80, 100],
                                    labels=["0-40", "41-60", "61-80", "81-100"])
disputes["response_bucket"] = pd.cut(disputes["response_time_days"], bins=[-1, 2, 7, 14, 30],
                                     labels=["0-2d", "3-7d", "8-14d", "15-30d"])

drivers = disputes.groupby(
    ["reason_code", "evidence_bucket", "response_bucket"],
    as_index=False,
    observed=True
).agg(

    disputes=("dispute_id", "count"),
    avg_amt=("dispute_amount_usd", "mean"),
    win_rate=("outcome", lambda x: (x == "WIN").mean())
).sort_values("disputes", ascending=False)

drivers.to_csv(os.path.join(OUT_DIR, "win_loss_drivers_slices.csv"), index=False)

# 3.4 Cohort analysis (merchant tenure cohorts + risk tier)
# Create tenure cohorts
def tenure_cohort(m):
    if m < 6:
        return "0-5m"
    if m < 12:
        return "6-11m"
    if m < 24:
        return "12-23m"
    if m < 36:
        return "24-35m"
    return "36m+"

mm["tenure_cohort"] = mm["tenure_months"].apply(tenure_cohort)

cohort = mm.groupby(["risk_tier", "tenure_cohort"], as_index=False).agg(
    merchants=("merchant_id", "nunique"),
    txn_count=("txn_count", "sum"),
    disputes=("dispute_count", "sum"),
    txn_amount=("txn_amount", "sum"),
    net_loss_usd=("net_loss_usd", "sum")
)

cohort["chargeback_rate"] = cohort["disputes"] / cohort["txn_count"]
cohort["net_loss_rate"] = np.where(cohort["txn_amount"] > 0, cohort["net_loss_usd"] / cohort["txn_amount"], 0)

cohort.to_csv(os.path.join(OUT_DIR, "cohort_risk_summary.csv"), index=False)

# ---------------------------
# 4) Charts
# ---------------------------

# 4.1 Overall monthly chargeback rate trend
overall_txn = transactions.groupby("month", as_index=False).agg(txn_count=("txn_id", "count"))
overall_disp = disputes.groupby("month", as_index=False).agg(disputes=("dispute_id", "count"))
overall = overall_txn.merge(overall_disp, on="month", how="left").fillna(0)
overall["chargeback_rate"] = overall["disputes"] / overall["txn_count"]

plt.figure()
plt.plot(pd.to_datetime(overall["month"] + "-01"), overall["chargeback_rate"])
plt.title("Overall Chargeback Rate Trend (Monthly)")
plt.xlabel("Month")
plt.ylabel("Chargeback Rate")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "overall_chargeback_rate_trend.png"), dpi=200)
plt.close()

# 4.2 Reason code volume bar
plt.figure()
plt.bar(reason_overall["reason_code"], reason_overall["disputes"])
plt.title("Dispute Volume by Reason Code")
plt.xlabel("Reason Code")
plt.ylabel("Disputes")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "dispute_volume_by_reason.png"), dpi=200)
plt.close()

# 4.3 Win rate by evidence bucket
win_ev = disputes.groupby("evidence_bucket", as_index=False).agg(win_rate=("outcome", lambda x: (x == "WIN").mean()),
                                                                disputes=("dispute_id", "count"))
plt.figure()
plt.plot(win_ev["evidence_bucket"].astype(str), win_ev["win_rate"], marker="o")
plt.title("Win Rate by Evidence Quality Bucket")
plt.xlabel("Evidence Quality Bucket")
plt.ylabel("Win Rate")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "win_rate_by_evidence_bucket.png"), dpi=200)
plt.close()

# 4.4 Heatmap-ish table (cohort chargeback rate)
pivot_cb = cohort.pivot(index="tenure_cohort", columns="risk_tier", values="chargeback_rate")
pivot_cb.to_csv(os.path.join(OUT_DIR, "cohort_chargeback_rate_pivot.csv"))

# Simple table render as image (no seaborn)
plt.figure()
plt.axis("off")
tbl = plt.table(cellText=np.round(pivot_cb.fillna(0).values, 4),
                rowLabels=pivot_cb.index.tolist(),
                colLabels=pivot_cb.columns.tolist(),
                loc="center")
tbl.auto_set_font_size(False)
tbl.set_fontsize(9)
tbl.scale(1, 1.4)
plt.title("Chargeback Rate by Tenure Cohort x Risk Tier")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "cohort_chargeback_rate_table.png"), dpi=200)
plt.close()

print("\nSaved outputs:")
print(" - outputs/merchant_month_risk_metrics.csv")
print(" - outputs/dispute_reason_overall.csv")
print(" - outputs/dispute_reason_by_tier.csv")
print(" - outputs/win_loss_drivers_slices.csv")
print(" - outputs/cohort_risk_summary.csv")
print(" - outputs/overall_chargeback_rate_trend.png")
print(" - outputs/dispute_volume_by_reason.png")
print(" - outputs/win_rate_by_evidence_bucket.png")
print(" - outputs/cohort_chargeback_rate_table.png")

# Quick sanity snapshots
print("\nTop 10 reason codes:")
print(reason_overall.head(10).to_string(index=False))

print("\nCohort summary (sample):")
print(cohort.head(10).to_string(index=False))


ROOT: /content
DATA_DIR: /content/data
OUT_DIR: /content/outputs

Saved raw data:
 - data/merchants.csv
 - data/transactions.csv
 - data/disputes.csv

Counts => merchants=250 txns=120,000 disputes=2,038


/tmp/ipython-input-474193161.py:355: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  win_ev = disputes.groupby("evidence_bucket", as_index=False).agg(win_rate=("outcome", lambda x: (x == "WIN").mean()),



Saved outputs:
 - outputs/merchant_month_risk_metrics.csv
 - outputs/dispute_reason_overall.csv
 - outputs/dispute_reason_by_tier.csv
 - outputs/win_loss_drivers_slices.csv
 - outputs/cohort_risk_summary.csv
 - outputs/overall_chargeback_rate_trend.png
 - outputs/dispute_volume_by_reason.png
 - outputs/win_rate_by_evidence_bucket.png
 - outputs/cohort_chargeback_rate_table.png

Top 10 reason codes:
     reason_code  disputes  dispute_amount  win_rate
           FRAUD       588        29300.80  0.340136
NOT_AS_DESCRIBED       375        19415.17  0.312000
    NOT_RECEIVED       371        18277.29  0.385445
         NO_AUTH       325        14096.75  0.384615
       DUPLICATE       149         7861.15  0.543624
PROCESSING_ERROR       136         6914.71  0.411765
       CANCELLED        94         5908.19  0.361702

Cohort summary (sample):
risk_tier tenure_cohort  merchants  txn_count  disputes  txn_amount  net_loss_usd  chargeback_rate  net_loss_rate
     High          0-5m          